In [23]:
import pandas as pd
import os
project_dir = os.getcwd()

In [24]:
icd = pd.read_csv(f"{project_dir}/original_data/hosp/d_icd_diagnoses.csv.gz")
icd.head()


,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [25]:
diagnoses = pd.read_csv(f"{project_dir}/original_data/hosp/diagnoses_icd.csv.gz")
diagnoses.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [26]:
import simple_icd_10_cm as cm

icd = icd[icd.icd_version == 10]
for i, row in icd.iterrows():
    if cm.is_valid_item(row.icd_code):
        chapter = cm.get_ancestors(row.icd_code)[-1]
        chapter_description = cm.get_description(chapter)
        icd.loc[i, "chapter"] = chapter + " - " + chapter_description
        block = cm.get_ancestors(row.icd_code)[-2]
        block_description = cm.get_description(block)
        icd.loc[i, "block"] = block_description


KeyboardInterrupt: 

In [11]:
icd.head()

,icd_code,icd_version,long_title,chapter,block
12238,A00,10,Cholera,1 - Certain infectious and parasitic diseases ...,Intestinal infectious diseases (A00-A09)
12239,A000,10,"Cholera due to Vibrio cholerae 01, biovar chol...",1 - Certain infectious and parasitic diseases ...,Intestinal infectious diseases (A00-A09)
12240,A001,10,"Cholera due to Vibrio cholerae 01, biovar eltor",1 - Certain infectious and parasitic diseases ...,Intestinal infectious diseases (A00-A09)
12241,A009,10,"Cholera, unspecified",1 - Certain infectious and parasitic diseases ...,Intestinal infectious diseases (A00-A09)
12242,A01,10,Typhoid and paratyphoid fevers,1 - Certain infectious and parasitic diseases ...,Intestinal infectious diseases (A00-A09)


In [14]:
    icd_code_count = (
        diagnoses[diagnoses.icd_version == 10]
        .groupby("icd_code")
        .agg({"subject_id":"count"})
        .sort_values("subject_id", ascending=False)
        .reset_index()
        .rename(columns={"subject_id":"icd_occurence"})
        )
    
    # Obtaining the ICD codes that occur at least 5k times in the datasets when assigned to patients
    # We then subset the dataset to contain those chosen ICD codes.
    # In this instance we only want 5 ICD codes.
    subjects = []
    while len(subjects) < 1000:
        # logging.info("Re-sampling due to small sample set")
        chosen_codes = icd_code_count[(icd_code_count.icd_occurence > 5000) & (icd_code_count.icd_code.str.len() >= 4)].sample(5).icd_code.tolist()
        df = icd[icd.icd_code.isin(chosen_codes)].merge(diagnoses, how="left", on=["icd_code", "icd_version"])
        subjects = df.subject_id.unique().tolist()

In [16]:
len(subjects)

29283

In [19]:
# emr = pd.DataFrame()
# for chunk in pd.read_csv(f"{project_dir}/original_data/hosp/emar.csv.gz", chunksize=100):
#     subbed = chunk[chunk.subject_id.isin(subjects)]
#     emr =  emr._append(subbed[~subbed.hadm_id.isna()])
emr.head(2)

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,enter_provider_id,charttime,medication,event_txt,scheduletime,storetime
190,10000117,27988844.0,10000117-20,20,10000117-53,NaN,NaN,2183-09-18 12:35:00,Influenza Vaccine Quadrivalent,Hold Dose,2183-09-18 12:35:00,2183-09-18 12:39:00
191,10000117,27988844.0,10000117-21,21,10000117-74,6506219.0,NaN,2183-09-18 22:23:00,Sodium Chloride 0.9% Flush,Flushed,2183-09-18 22:23:00,2183-09-18 22:23:00


In [27]:
    labitems = pd.read_csv(f"{project_dir}/original_data/hosp/d_labitems.csv.gz")
    labevents = pd.DataFrame()
    for idx, chunk in enumerate(pd.read_csv(f"{project_dir}/original_data/hosp/labevents.csv.gz", chunksize=100, low_memory=False)):
        subbed = chunk[chunk["subject_id"].isin(subjects)][["labevent_id", "subject_id", "hadm_id", "itemid", "flag", "priority", "comments"]]
        if len(subbed) > 0:
            subbed = (
                subbed
                .merge(labitems, on="itemid", how="left")
                )

KeyboardInterrupt: 

In [34]:
    admission = pd.read_csv(f"{project_dir}/original_data/hosp/admissions.csv.gz")


In [35]:
admission.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
